In [1]:
import numpy as np
import pandas as pd
import os
import glob

# Load and create DataFrame

### Load and combine files into one DataFrame

In [54]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(os.path.dirname(os.path.abspath('')),"raw_data","api_test1", "*.csv"))

# Define column names 
col_names=['icao24','callsign','origin_country','time_position','last_contact','long','lat','baro_altitude','on_ground','velocity',       
'true_track','vertical_rate','sensors','geo_altitude','squawk','spi','position_source', "drop", "time"]

# Concatenate to large Dataframe
df = pd.concat((pd.read_csv(f, index_col=0, names=col_names, header=0) for f in all_files)) 
df.drop(columns=["drop"], inplace=True) # drop column where we don't know what i represents (not mentioned in Documentation)
df.dropna(subset=['lat', "long", "icao24"], inplace=True) #drop rows where lat or long or icao24 is NaN


### Function to Filter for passenger flights 

In [55]:
# Create function to filter dataframe for passenger carriers based on Wikipedia List

def filter_passenger_carriers(flights_df):
    
    # Create list of carriers and type of transport 
    wikiurl = "https://de.wikipedia.org/wiki/Liste_von_Fluggesellschaften" #URL of lists of carriers from 
    carriers_df = pd.DataFrame(pd.read_html(wikiurl, keep_default_na=False)[1]) # read table from wikipedia
    car_types = ["B", "C", "P", "P+", "U", "U+"] # Labels for passenger carriers
    p_carriers = carriers_df[carriers_df["Bemerkung"].isin(car_types)] # Filter wikipedia table for passenger carriers

    # merge carrier to df
    pflights_df = flights_df[flights_df.callsign.str[:3].isin(p_carriers.ICAO)].copy() # Filter flights only to include passenger flights
    pflights_df["callsign_carrier"] = pflights_df["callsign"].str[:3] # extra column (not sure if needed)
    pflights_df = pflights_df.merge(p_carriers[["Name", "ICAO"]], left_on="callsign_carrier", right_on="ICAO", how="left")
    pflights_df.rename(columns = {'Name':'carrier_company', "ICAO":"icao"}, inplace = True)
    pflights_df.drop(columns=["icao"], inplace=True)
    
    return pflights_df
    

In [56]:
pflights_df = filter_passenger_carriers(df)

In [57]:
pflights_df.head()

,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,time,callsign_carrier,carrier_company
0,a808c4,PDT6104,United States,1652824461.0,1.652824e+09,-72.5089,42.8871,7315.20,0.0,193.95,280.24,-0.33,NaN,7223.76,5354.0,0.0,0,1652824461,PDT,Piedmont Airlines
1,4b1816,SWR240J,Switzerland,1652824461.0,1.652824e+09,17.6831,58.9641,4221.48,0.0,182.06,24.90,-10.08,NaN,4343.40,3076.0,0.0,0,1652824461,SWR,Swiss
2,ab1644,UAL1955,United States,1652824461.0,1.652824e+09,-96.2348,29.5126,4427.22,0.0,167.52,42.26,-11.05,NaN,4686.30,NaN,0.0,0,1652824461,UAL,United Airlines
3,e8027c,LPE2314,Chile,1652824461.0,1.652824e+09,-77.4207,-12.0136,3703.32,0.0,185.91,316.01,20.16,NaN,NaN,NaN,0.0,0,1652824461,LPE,LATAM Airlines Perú
4,aa56db,UAL2496,United States,1652824455.0,1.652824e+09,-94.8009,29.9033,4008.12,0.0,178.81,95.45,6.83,NaN,4213.86,NaN,0.0,0,1652824461,UAL,United Airlines


### Function to merge aircraft data to df

In [157]:
def get_aircraft_data(pflights_df):
    # Load Database from OpenSky (I downloaded it from their website, but it gets currently updated)
    opensky_DB = pd.read_csv("../raw_data/aircraftDatabase.csv")

    # Merge pflights df with openskyDB on "icao24"
    pflights_df = pflights_df.merge(opensky_DB[['icao24', 'registration', 'manufacturericao', 'manufacturername',
           'model', 'typecode', 'serialnumber', 'linenumber', 'icaoaircrafttype',
           'operator', 'operatorcallsign', 'operatoricao', 'operatoriata', 'owner']], on="icao24", how="left")
    
    return pflights_df

In [58]:
pflights_df = get_aircraft_data(pflights_df)

### Function for Reverse Geoencoding

In [60]:
# Get location based on lat and long
import reverse_geocoder

In [154]:
# Function to map cities based on lat and long
def rev_geocode(pflights_df):
    
    # create coord column with the required format for the reverse_geocoder
    pflights_df["coord"]= [(lat, long) for lat, long in zip(pflights_df["lat"], pflights_df["long"])]
    
    # Get coordinates 
    coordinates = pflights_df.coord.tolist()
    rev_geo = reverse_geocoder.search(coordinates)
    
    # Create df and filter important rows
    rev_geo_df = pd.DataFrame(rev_geo)[["name", "admin1", "admin2", "cc"]]
    rev_geo_df.columns = ["city_name", "reg_admin1", "reg_admin2", "country_cc"]
    
    # Concat Geo Data with old dataframe
    pflights_df_final = pd.concat([pflights_df, rev_geo_df], axis=1)
    
    return pflights_df_final

In [78]:
# Concat Geo Data with old dataframe
pflights_df_f = rev_geocode(pflights_df)

### Create combined preprocessing function

In [158]:
def preproc_flight_data(df):
    flights_df = filter_passenger_carriers(df)
    flights_df = get_aircraft_data(flights_df)
    flights_df = rev_geocode(flights_df)
    
    return flights_df

In [159]:
pflights_df_f=preproc_flight_data(df)

In [160]:
pflights_df_f.shape

(159522, 38)

# Find aircrafts which have been on the ground in two different cities

In [165]:
# find aircrafts who had the status on ground True AND False
groups = pflights_df_f.groupby("icao24")[["on_ground", "time"]].nunique()#.sort_values(by=["on_ground"], ascending=False)
land_air = groups[groups['on_ground'] == 2].sort_values(by=["time"], ascending=False)
land_air

,on_ground,time
icao24,,
4075e0,2,44
4ca92d,2,44
4074b5,2,44
4b1808,2,44
407ac4,2,44
...,...,...
45caa6,2,2
7c6b39,2,2
71bf72,2,2


In [169]:
#Filter original df for records from airplanes which have been on ground an in the air
land_air_df = pflights_df_f[pflights_df_f.icao24.isin(land_air.index)]

In [172]:
land_air_df.sort_values(by=["icao24","time"], ascending=False).head()

,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,on_ground,velocity,...,operator,operatorcallsign,operatoricao,operatoriata,owner,coord,city_name,reg_admin1,reg_admin2,country_cc
55622,e8045b,LPE2485,Chile,1652833566.0,1.652834e+09,-8.1964,39.0654,10058.40,0.0,229.87,...,NaN,LAN,LAN,NaN,Latam Airlines,"(39.0654, -8.1964)",Santo Andre,Portalegre,Ponte de Sor,PT
40820,e8045b,LPE2485,Chile,1652832665.0,1.652833e+09,-5.9398,39.7388,8724.90,0.0,243.19,...,NaN,LAN,LAN,NaN,Latam Airlines,"(39.7388, -5.9398)",Torrejon el Rubio,Extremadura,Provincia de Caceres,ES
8766,e8045b,LPE2485,Chile,1652832362.0,1.652832e+09,-5.1853,40.0590,7475.22,0.0,240.10,...,NaN,LAN,LAN,NaN,Latam Airlines,"(40.059, -5.1853)",Navalcan,Castille-La Mancha,Province of Toledo,ES
95599,e8045b,LPE2485,Chile,1652831764.0,1.652832e+09,-3.8129,40.6219,3665.22,0.0,174.22,...,NaN,LAN,LAN,NaN,Latam Airlines,"(40.6219, -3.8129)",Colmenar Viejo,Madrid,Provincia de Madrid,ES
150044,e8045b,LPE2485,Chile,1652831457.0,1.652831e+09,-3.5750,40.5557,891.54,0.0,108.58,...,NaN,LAN,LAN,NaN,Latam Airlines,"(40.5557, -3.575)",San Sebastian de los Reyes,Madrid,Provincia de Madrid,ES


In [228]:
# Find airplanes which have been on the ground in two different cities
full_trip = land_air_df.groupby(["icao24", "on_ground"])[["reg_admin1"]].nunique().query('(on_ground==1)&(reg_admin1>1)')

In [229]:
# Number of airplanes which have been on grund in two different cities
full_trip.shape[0]

313

In [230]:
#Filter original df for records from airplanes which have been on ground in two different cities
full_trip_df= pflights_df_f[pflights_df_f.icao24.isin(full_trip.index.get_level_values("icao24"))].sort_values(by=["icao24","time"], ascending=False)
full_trip_df

,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,on_ground,velocity,...,operator,operatorcallsign,operatoricao,operatoriata,owner,coord,city_name,reg_admin1,reg_admin2,country_cc
36732,c8237a,ANZ827M,New Zealand,1652862078.0,1.652862e+09,175.0014,-40.9377,4145.28,0.0,150.53,...,NaN,NaN,NaN,NaN,Air New Zealand Link,"(-40.9377, 175.0014)",Paraparaumu,Wellington,Kapiti Coast District,NZ
72947,c8237a,ANZ827M,New Zealand,1652859768.0,1.652860e+09,175.3605,-38.1232,2674.62,0.0,104.53,...,NaN,NaN,NaN,NaN,Air New Zealand Link,"(-38.1232, 175.3605)",Otorohanga,Waikato,Otorohanga District,NZ
127468,c8237a,ANZ827M,New Zealand,1652859697.0,1.652860e+09,175.3644,-38.0568,2186.94,0.0,100.42,...,NaN,NaN,NaN,NaN,Air New Zealand Link,"(-38.0568, 175.3644)",Cambridge,Waikato,Waipa District,NZ
54658,c8237a,ANZ785M,New Zealand,1652833567.0,1.652834e+09,175.8965,-39.7474,4876.80,0.0,130.68,...,NaN,NaN,NaN,NaN,Air New Zealand Link,"(-39.7474, 175.8965)",Waiouru,Manawatu-Wanganui,Ruapehu District,NZ
39864,c8237a,ANZ785M,New Zealand,1652832666.0,1.652833e+09,176.2091,-38.7438,4556.76,0.0,105.61,...,NaN,NaN,NaN,NaN,Air New Zealand Link,"(-38.7438, 176.2091)",Taupo,Waikato,Taupo District,NZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110602,040128,ETH687,Ethiopia,1652821661.0,1.652822e+09,77.0809,28.5649,NaN,1.0,0.00,...,Ethiopian Airlines,ETHIOPIAN,ETH,ET,Ethiopian Airlines,"(28.5649, 77.0809)",Nangloi Jat,NCT,West Delhi,IN
81432,040128,ETH687,Ethiopia,1652821331.0,1.652821e+09,77.0823,28.5631,NaN,1.0,2.31,...,Ethiopian Airlines,ETHIOPIAN,ETH,ET,Ethiopian Airlines,"(28.5631, 77.0823)",Nangloi Jat,NCT,West Delhi,IN
63516,040128,ETH687,Ethiopia,1652821090.0,1.652821e+09,77.0833,28.5618,NaN,1.0,0.90,...,Ethiopian Airlines,ETHIOPIAN,ETH,ET,Ethiopian Airlines,"(28.5618, 77.0833)",Gurgaon,Haryana,Gurgaon,IN
132232,040128,ETH687,Ethiopia,1652821028.0,1.652821e+09,77.0836,28.5614,NaN,1.0,0.26,...,Ethiopian Airlines,ETHIOPIAN,ETH,ET,Ethiopian Airlines,"(28.5614, 77.0836)",Gurgaon,Haryana,Gurgaon,IN


### Plot complete route 

In [231]:
# plot one flight from full_trip_df
flight_test = pflights_df_f[pflights_df_f.icao24=="aaa5b8"]

# Plot flight route
import plotly.express as px
fig = px.scatter_geo(flight_test, lat='lat',lon='long', hover_name="icao24", color="geo_altitude", hover_data=["on_ground", "time"])
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

flight_test[flight_test.geo_altitude.isnull()].sort_values(by=["time"])\
    .reset_index(drop=True).iloc[[0,-1]]\
    [["icao24", "time", "city_name", "reg_admin1", "country_cc"]]

,icao24,time,city_name,reg_admin1,country_cc
0,aaa5b8,1652820761,SeaTac,Washington,US
10,aaa5b8,1652830554,San Leandro,California,US


Flight from SeaTac (Seattle) to Washington

In [239]:
# Plot flight route of first 50 flights from full_flights
df_plot = pflights_df_f[pflights_df_f.icao24.isin(full_trip.index.get_level_values("icao24")[:50])]

fig = px.scatter_geo(df_plot, lat='lat',lon='long', hover_name="icao24", color="geo_altitude", hover_data=["on_ground", "time"])
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [238]:
pflights_df_f[pflights_df_f.icao24=="06a053"].sort_values(by=["time"])[["icao24", "time", "on_ground","geo_altitude","city_name", "reg_admin1", "country_cc"]]

,icao24,time,on_ground,geo_altitude,city_name,reg_admin1,country_cc
53380,06a053,1652821965,0.0,11978.64,Riyadh,Ar Riyad,SA
45737,06a053,1652822273,0.0,11971.02,Riyadh,Ar Riyad,SA
126189,06a053,1652822597,0.0,11734.80,Riyadh,Ar Riyad,SA
60536,06a053,1652822921,0.0,8214.36,Al Hufuf,Eastern Province,SA
6548,06a053,1652823534,0.0,6766.56,Umm Bab,,QA
158614,06a053,1652823854,0.0,3710.94,Umm Bab,,QA
49508,06a053,1652824158,0.0,1844.04,Umm Sa'id,Al Wakrah,QA
2782,06a053,1652824461,0.0,800.10,Al Wakrah,Al Wakrah,QA
68265,06a053,1652824770,1.0,NaN,Doha,Baladiyat ad Dawhah,QA
100418,06a053,1652825076,1.0,NaN,Doha,Baladiyat ad Dawhah,QA


- There seems to be a problem identifying flights which started and landed based on region and geo_atitude (especially in Qatar, probably because it is so close to the border?)
- Other ways need to be found
- It has to be clarified, how we can distinguish different flights from the same aircraft 
    - (Is it  "removed" from the api until the next start?)
    - Is the callsign different ? 